In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import warnings
import mlflow
from modeling.config import EXPERIMENT_NAME
TRACKING_URI = open("../.mlflow_uri").read().strip()

ROOT = os.environ.get('PWD')

warnings.filterwarnings('ignore')

# our data, read into dataframe df_master_tfw
df_master_tfw = pd.read_csv('../data/master_no_missing_20210719.csv')





In [2]:
df_master_tfw.head()

,Unnamed: 0,listing_id,1_room_apartment,library,children`s_room,corridor,dining_room,en_suite_bathroom,gallery,heating_room,...,pool,sauna,grill,garden,terrace,balcony,bathrooms,bedrooms,max_guests,living_area
0,0,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1,1,1,1,0,1,1.0,2.0,4.0,60.0
1,1,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1,1,1,1,0,1,1.0,2.0,4.0,60.0
2,2,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1,1,1,1,0,1,1.0,2.0,4.0,60.0
3,3,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1,1,1,1,0,1,1.0,2.0,4.0,60.0
4,4,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1,1,1,1,0,1,1.0,2.0,4.0,60.0


# Data cleaning and feature engineering

In [3]:
del df_master_tfw['Unnamed: 0']

In [4]:
list(df_master_tfw.columns.values)

['listing_id',
 '1_room_apartment',
 'library',
 'children`s_room',
 'corridor',
 'dining_room',
 'en_suite_bathroom',
 'gallery',
 'heating_room',
 'kitchen',
 'kitchen_living',
 'living_bedroom',
 'living_/_dining_room',
 'living_room',
 'lounge',
 'roof-deck',
 'separate_WC',
 'storage_room',
 'veranda',
 'washroom',
 'wellness',
 'winter_garden',
 'Blu-ray_player',
 'CDs_DVDs',
 'DVD-player',
 'VCR',
 'additional_bed',
 'alarm_system',
 'armchair',
 'awning_',
 'baby_monitor',
 'bath_towels',
 'bathrobe',
 'bathtub',
 'beach_chair',
 'bed_linen',
 'bicycles',
 'bidet',
 'boat',
 'boat_dock',
 'books',
 'bottle_warmer',
 'bread_service',
 'bunk_bed',
 'carport',
 'central_heating',
 'changing_mat',
 'chest_of_drawers',
 "child's_bed",
 'children_toilet_seat',
 'cleaning_supplies',
 'coffee_machine',
 'colouring_book_pencils',
 'computer',
 'crockery',
 'darts',
 'desk',
 'dining_table',
 'double_bed',
 'double_wash_basin',
 'drying_rack',
 'egg_cooker',
 'exercise_room',
 'fan',
 'f

In [5]:
# set display default so that we can see all the columns we have in play

pd.set_option('display.max_rows', None)

In [6]:
# convert to datetime, this list can be varied depending on what stays in for a particular model

dates = ['arrival_date', 'departure_date', 'date']

for i in dates:
    df_master_tfw[i] = pd.to_datetime(df_master_tfw[i])

In [7]:


df_master_tfw.dtypes

listing_id                                object
1_room_apartment                         float64
library                                  float64
children`s_room                          float64
corridor                                 float64
dining_room                              float64
en_suite_bathroom                        float64
gallery                                  float64
heating_room                             float64
kitchen                                  float64
kitchen_living                           float64
living_bedroom                           float64
living_/_dining_room                     float64
living_room                              float64
lounge                                   float64
roof-deck                                float64
separate_WC                              float64
storage_room                             float64
veranda                                  float64
washroom                                 float64
wellness            

In [21]:
# calculate overall number of inquiries per property

df_master_tfw['total_inquiries_per_property'] = df_master_tfw.groupby('listing_id')['title'].count().reset_index()['title']



In [22]:
df_master_tfw.head(10)

,listing_id,1_room_apartment,library,children`s_room,corridor,dining_room,en_suite_bathroom,gallery,heating_room,kitchen,...,grill,garden,terrace,balcony,bathrooms,bedrooms,max_guests,living_area,total_id_per_property,total_inquiries_per_property
0,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,1,0,1,1.0,2.0,4.0,60.0,NaN,23.0
1,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,1,0,1,1.0,2.0,4.0,60.0,NaN,310.0
2,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,1,0,1,1.0,2.0,4.0,60.0,NaN,24.0
3,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,1,0,1,1.0,2.0,4.0,60.0,NaN,8.0
4,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,1,0,1,1.0,2.0,4.0,60.0,NaN,7.0
5,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,1,0,1,1.0,2.0,4.0,60.0,NaN,24.0
6,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,1,0,1,1.0,2.0,4.0,60.0,NaN,25.0
7,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,1,0,1,1.0,2.0,4.0,60.0,NaN,35.0
8,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,1,0,1,1.0,2.0,4.0,60.0,NaN,58.0
9,97232bc1-cee6-54cc-9965-be13177051d3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,1,0,1,1.0,2.0,4.0,60.0,NaN,8.0


In [23]:
df_master_tfw['total_inquiries_per_property'].mean()

nan

In [17]:
df_master_tfw['total_id_per_property'].unique()

array([ 212,  172,  252,  318,   22,    1,    2,    4,    3,   13,    8,
          6,    5,   10,    7,    9,   11,   12,   24,   14,   28,   16,
         20,   17,   15,   30,   38,   43,   37,   47,   25,   44,   23,
         21,   19,   33,   45,   40,   39,   27,   34,   26,   50,   49,
         59,   31,   84,   53,   64,   18,   52,   29,   35,   58,   42,
         75,   57,   36,   99,  535,   70,   32,   71,  128,   51,   66,
         72,  117,  131,   41,  134,  190,   76,   60,  144,  191,  468,
        240,   65,  120,  102,   55,   81,  106,   61,   54,  104,   90,
         85,  130,   48,   86,   87,  100,  124,   79,   78,   73,   77,
         67,   74,   63,  139,  103,  151,  179,  105,  145,   68, 1055,
         56, 1110,   62,  169,   82,   91,   83,   80,   46,  101,  133,
        153,   92,   95,  113,  201,  158,  132,  164,  141,  107,  110,
        140,  215,   97,  109,  242,   88,  165,  426,  250,  184,   69,
        163,  114,  154,  377,  119,   98,  116, 12

## Splitting data for testing 

In [18]:
smaller 
X = df_master_tfw.drop('listing_id','inquiry_count', axis=1)
Y = df_master_tfw['total_id_per_property']

TypeError: drop() got multiple values for argument 'axis'

In [10]:
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [11]:
#dropping Quakers column and unnamed
#changing one of the altitude to log and droping the original
X_train["altitude_mean_log"] = np.log(X_train["altitude_mean_meters"])
X_train.drop(['altitude_mean_meters'], axis=1, inplace=True)
X_train.drop(['Quakers'], axis=1, inplace=True)
X_train.drop(['Unnamed: 0'], axis=1, inplace=True)

In [12]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 17 to 6
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Number.of.Bags        22 non-null     int64  
 1   Harvest.Year          22 non-null     int64  
 2   Fragrance...Aroma     22 non-null     float64
 3   Flavor                22 non-null     float64
 4   Aftertaste            22 non-null     float64
 5   Salt...Acid           22 non-null     float64
 6   Bitter...Sweet        22 non-null     float64
 7   Mouthfeel             22 non-null     float64
 8   Uniform.Cup           22 non-null     float64
 9   Clean.Cup             22 non-null     float64
 10  Balance               22 non-null     float64
 11  Cupper.Points         22 non-null     float64
 12  Total.Cup.Points      22 non-null     float64
 13  Moisture              22 non-null     float64
 14  Category.One.Defects  22 non-null     int64  
 15  Category.Two.Defects  22 

In [14]:
# fillna with mean.. 
X_train["altitude_low_meters"] = X_train["altitude_low_meters"].fillna(altitude_low_meters_mean)
X_train["altitude_high_meters"] = X_train["altitude_high_meters"].fillna(altitude_high_meters_mean)
X_train["altitude_mean_log"] = X_train["altitude_mean_log"].fillna(altitude_mean_log_mean)

In [15]:
print(f"altitude low meters mean is {altitude_low_meters_mean}")
print(f"altitude_high_meters_mean is {altitude_high_meters_mean}")
print(f"altitude_mean_log_mean is {altitude_mean_log_mean}")

altitude low meters mean is 1500.3684210526317
altitude_high_meters_mean is 1505.6315789473683
altitude_mean_log_mean is 7.0571530664031155


## Trainining the model and tracking with MLFlow

In [16]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.start_run()
run = mlflow.active_run()

In [17]:
print("Active run_id: {}".format(run.info.run_id))

Active run_id: db36ca6e38d84e31ac34a75bfa9b9838


In [22]:
#training the model
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)

In [23]:
from sklearn.metrics import mean_squared_error
y_train_pred = reg.predict(X_train)
mse_train = mean_squared_error(y_train, y_train_pred)
print(mse_train)

6.701014816713759e-28


In [24]:
#dropping Quakers column and unnamed
#changing one of the altitude to log and droping the original
X_test["altitude_mean_log"] = np.log(X_test["altitude_mean_meters"])
X_test.drop(['altitude_mean_meters'], axis=1, inplace=True)
X_test.drop(['Quakers'], axis=1, inplace=True)
X_test.drop(['Unnamed: 0'], axis=1, inplace=True)
# fillna with mean.. 
X_test["altitude_low_meters"] = X_test["altitude_low_meters"].fillna(altitude_low_meters_mean)
X_test["altitude_high_meters"] = X_test["altitude_high_meters"].fillna(altitude_high_meters_mean)
X_test["altitude_mean_log"] = X_test["altitude_mean_log"].fillna(altitude_mean_log_mean)

KeyError: 'altitude_mean_meters'

In [25]:
y_test_pred = reg.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_pred)
print(mse_test)

2.08680004794465e-27


In [26]:
params = {
      "altitude_low_meters_mean": altitude_low_meters_mean,
      "altitude_high_meters_mean": altitude_high_meters_mean,
      "altitude_mean_log_mean": altitude_mean_log_mean,
      "fit_intercept": True,
  }

In [27]:
mlflow.log_params(params)
mlflow.set_tag("running_from_jupyter", "made myself crazy")
mlflow.log_metric("train -" + "MSE", mse_train)
mlflow.log_metric("test -" + "MSE", mse_test)
# mlflow.log_artifact("../models")
# mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [28]:
mlflow.get_run(run_id=run.info.run_id)

<Run: data=<RunData: metrics={'test -MSE': 2.08680004794465e-27, 'train -MSE': 6.701014816713759e-28}, params={'altitude_high_meters_mean': '1505.6315789473683',
 'altitude_low_meters_mean': '1500.3684210526317',
 'altitude_mean_log_mean': '7.0571530664031155',
 'fit_intercept': 'True'}, tags={'mlflow.source.name': 'ipykernel_launcher',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'petermcisaac',
 'running_from_jupyter': 'True'}>, info=<RunInfo: artifact_uri='s3://neuefische-mlflow/mlflow-artifacts/25/db36ca6e38d84e31ac34a75bfa9b9838/artifacts', end_time=1626427388226, experiment_id='25', lifecycle_stage='active', run_id='db36ca6e38d84e31ac34a75bfa9b9838', run_uuid='db36ca6e38d84e31ac34a75bfa9b9838', start_time=1626427268688, status='FINISHED', user_id='petermcisaac'>>

## Checking the experiments

while the next cell is running you will not be able to run other cells in the notebook

In [ ]:
!mlflow ui